# Experiment context

In [13]:
import os
print(os.getcwd())

/Users/yavuuzsameet/Desktop/polimi/tesi/recombinhunt-cov-7.0.0/fromzero


In [14]:
from recombinhunt.core.method import *
from recombinhunt.core.environment import Environment

env = Environment("../environments/env_nextstrain_2025_06_30")
custom_environments = {
    "XBB": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XBB") or any([sl in l for sl in ("EG", "EK", "EL", "EM", "EU", "FD", "FE", "FG", "FH", "FL")])]),
    "XBF": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XBF")]),
    "XAY": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XAY")]),
    "XP": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XBB")])
}

# Preparation of test cases

- generation of consensus sequence for each recombinant case
- collection of PANGO-suggested breakpoint positions and recombination candidates to compare with RecombinHunt's output

In [15]:
import json
from recombinhunt.core.environment import PangoLineageHierarchy
from recombinhunt.validation.utils import *

# GROUND TRUTH
lh = PangoLineageHierarchy("./validation_data/alias_key.json")
cl = AssessedContributingLin("./validation_data/alias_key.json")


# NUMBER OF SEQUENCES PER CASE
with open("./demo_input_nextstrain/samples_total.json") as inp:
    number_of_sequences_by_rec_case = json.load(inp)


In [16]:
# CASE CLASSIFICATION
cases_1BP = [l for l in number_of_sequences_by_rec_case.keys() if BreakpointsLocation.breakpoints_num(l) == 1 and BreakpointsLocation.ith_breakpoint(l, 0)[0] != 0]
cases_2BP = [l for l in number_of_sequences_by_rec_case.keys() if BreakpointsLocation.breakpoints_num(l) == 2 and BreakpointsLocation.ith_breakpoint(l, 0)[0] != 0]
# collect cases with 3+ breakpoints and inaccurate candidates/breakpoints
cases_oth = [l for l in number_of_sequences_by_rec_case.keys() if BreakpointsLocation.breakpoints_num(l) == 0 or BreakpointsLocation.breakpoints_num(l) > 2 or BreakpointsLocation.ith_breakpoint(l, 0)[0] == 0]

print(f"Cases with 1 breakpoint: \n{cases_1BP}")
print()
print(f"Cases with 2 breakpoints: \n{cases_2BP}")
print()
print(f"Cases with 3+ breakpoints or inaccurate candidates/breakpoints: \n{cases_oth}")

Cases with 1 breakpoint: 
['XAA', 'XAB', 'XAD', 'XAE', 'XAF', 'XAG', 'XAH', 'XAL', 'XAM', 'XAN', 'XAP', 'XAS', 'XAT', 'XAU', 'XAV', 'XB', 'XBB', 'XBD', 'XBE', 'XBF', 'XBG', 'XBH', 'XBJ', 'XBM', 'XBP', 'XBR', 'XBW', 'XE', 'XG', 'XH', 'XJ', 'XL', 'XM', 'XN', 'XQ', 'XS', 'XU', 'XW', 'XY', 'XZ']

Cases with 2 breakpoints: 
['XAC', 'XAK', 'XAZ', 'XBL', 'XBT', 'XBU', 'XD']

Cases with 3+ breakpoints or inaccurate candidates/breakpoints: 
['XAJ', 'XBK', 'XBN', 'XBV', 'XBZ', 'XCA', 'XCC', 'XCE', 'XCF', 'XCG', 'XCH', 'XCJ', 'XCK', 'XCL', 'XCM', 'XCP', 'XCQ', 'XCR', 'XCT', 'XCU', 'XCV', 'XCW', 'XCY', 'XCZ', 'XDA', 'XDB', 'XDC', 'XDD', 'XDE', 'XDG', 'XDH', 'XDK', 'XDL', 'XDN', 'XDP', 'XDQ', 'XDR', 'XDV', 'XDY', 'XEB', 'XEC', 'XED', 'XEE', 'XEF', 'XEG', 'XEH', 'XEK', 'XEL', 'XEM', 'XEN', 'XEP']


In [17]:
# TARGET OF CASE (consensus sequence)
case2target = dict()
for case in cases_1BP + cases_2BP + cases_oth:
    this_case_sequences_str = []
    with open(f"./demo_input_nextstrain/samples_{case}.csv") as inp:
        inp.readline()
        for line in inp.readlines():
            name, true_lin, nuc_changes = line.rstrip('\n').split('\t')
            this_case_sequences_str.append(nuc_changes)
    case2target[case] = ["avg75", case, compute_75_perc_characterization(strings=this_case_sequences_str)]


def group_of_rec_case(rec_lineage, target_seq: list):
    if BreakpointsLocation.breakpoints_num(rec_lineage) == 2:
        return "2BP"
    elif BreakpointsLocation.breakpoints_num(rec_lineage) == 1:
        t_br_start, t_br_end = BreakpointsLocation.to_target_pos(BreakpointsLocation.ith_breakpoint(rec_lineage, 0), target_seq)
        t_len = len(target_seq)
        if t_br_start >= 10 and t_br_end <= (t_len - 10):
            return "1BP mid"
        elif t_br_start < 10:
            return "1BP 5'"
        else:
            return "1BP 3'"
    else:
        return "?"

cases_1BP_mid = [c for c in cases_1BP if group_of_rec_case(c, case2target[c][-1]) == '1BP mid']
cases_1BP_5p = [c for c in cases_1BP if group_of_rec_case(c, case2target[c][-1]) == '1BP 5\'']
cases_1BP_3p = [c for c in cases_1BP if group_of_rec_case(c, case2target[c][-1]) == '1BP 3\'']


print(len(cases_1BP), len(cases_2BP), len(cases_oth))


40 7 51


In [19]:
output_dir = './experiment_results'
os.makedirs(output_dir, exist_ok=True)

old_lineages_txt_path = os.path.join(output_dir, 'sarscov2-experiment-results(old-lineages).txt')
new_lineages_txt_path = os.path.join(output_dir, 'sarscov2-experiment-results(new-lineages).txt')

old_lineages_html_path = os.path.join(output_dir, 'sarscov2-experiment-details(old-lineages).html')
new_lineages_html_path = os.path.join(output_dir, 'sarscov2-experiment-details(new-lineages).html')

old_lineages_comparison_path = os.path.join(output_dir, 'sarscov2-experiment-comparison(old-lineages).txt')
new_lineages_comparison_path = os.path.join(output_dir, 'sarscov2-experiment-comparison(new-lineages).txt')

# Run of experiments and validation

In [20]:
OLD_LINEAGES = ["XA", "XAD", "XAE", "XAL", "XAN", "XAP", "XAT", "XAV", "XBB", "XBD", "XBE", "XBF", "XBG", "XBH", "XBJ", "XBM", "XBR", "XBP", "XBW", "XJ", "XM", "XV", "XY", "XZ", "XAA", "XAB", "XAF", "XAG", "XAM", "XAR", "XAU", "XF", "XE", "XG", "XH", "XN", "XL", "XQ", "XR", "XS", "XW", "XU", "XAH", "XP", "XAC", "XAK", "XAZ", "XBL", "XBT", "XBU", "XD"]
length_of_old_lineages = len(OLD_LINEAGES)
length_of_old_lineages

51

In [21]:
with open('./validation_data/alias_key.json') as f:
    alias_key = json.load(f)

In [22]:
env.assert_valid()

Starting formal checks of the environment...


In [ ]:
from tqdm import tqdm
from recombinhunt.validation.case_analysis import CaseAnalysis

# CHOOSE ANALYSIS GROUP
#groups_of_cases, group_names, number_of_cases = (cases_2BP,), ("2BP",), len(cases_2BP)
#groups_of_cases, group_names, number_of_cases = (cases_1BP_mid, cases_1BP_5p, cases_1BP_3p), ("1BP_mid", "1BP_5p", "1BP_3p"), len(cases_1BP)
# groups_of_cases, group_names, number_of_cases = (cases_1BP_mid, cases_1BP_5p, cases_1BP_3p, cases_2BP), ("1BP_mid", "1BP_5p", "1BP_3p", "2BP"),  len(cases_1BP) + len(cases_2BP)
# groups_of_cases, group_names, number_of_cases = (cases_oth,), ("?",),  len(cases_oth)
groups_of_cases, group_names, number_of_cases = (cases_1BP_mid, cases_1BP_5p, cases_1BP_3p, cases_2BP, cases_oth), ("1BP mid", "1BP 5'", "1BP 3'", "2BP", "undefined"),  len(cases_1BP) + len(cases_2BP) + len(cases_oth)

# ANALYSIS  
issues = {
    "issue_0BP": [],    # cases labeled as recombinant by PANGO and non-recombinant by RecombinHunt
    "issue_2BP": [],    # cases labeled as 1BP recombinant by PANGO and 2BP by RecombinHunt
    "issue_1BP": [],    # 
    "issue_KO": [],
    "non_issue_OK": []
}

case_n = 0
met = False
with open(old_lineages_txt_path, 'w') as old_lineages_txt, \
     open(new_lineages_txt_path, 'w') as new_lineages_txt, \
     open(old_lineages_html_path, 'w') as old_lineages_html, \
     open(new_lineages_html_path, 'w') as new_lineages_html, \
     open(old_lineages_comparison_path, 'w') as old_lineages_comparison, \
     open(new_lineages_comparison_path, 'w') as new_lineages_comparison:
    with tqdm(total=number_of_cases) as progress:
        for group_of_cases, group_name in zip(groups_of_cases, group_names):
            for case in group_of_cases:
                try:
                    def log(msg, file):
                        file.write(str(msg) + '\n')

                    case_n += 1
                    progress.update()

                    # if not met and case != "XAV":
                    #     continue
                    # else:
                    #     met = True
                    #     pass
                    # if not case in ("XP"):
                    #     continue

                    # GROUND TRUTH DATA
                    N_seq = number_of_sequences_by_rec_case[case]
                    #parent_lineage_list = cl.contributing_to(case)
                    #breakpoints = BreakpointsLocation.all_breakpoints(case)
                    #n_breakpoints = BreakpointsLocation.breakpoints_num(case)
                    nuc_changes = case2target[case][-1]

                    # EXPERIMENT
                    exp = Experiment(custom_environments.get(case, env))
                    exp.set_target(nuc_changes)
                    result = exp.run()

                    if result and hasattr(result, 'genome_view') and result.genome_view and \
                        hasattr(result.genome_view, 'describe') and callable(result.genome_view.describe):
                        description = result.genome_view.describe()
                        recombinhunt_candidates = description.get('designated candidates', [])

                    #print(f"Analysing case {case} ({case_n}/{number_of_cases}) with {N_seq} sequences, {n_breakpoints} breakpoints, group: {group_name}")
                    print(f"Analysing case {case} ({case_n}/{number_of_cases}) with {N_seq} sequences, group: {group_name}")
                    
                    if case in OLD_LINEAGES:
                        log('------------------------', old_lineages_txt)
                        log(f"{case} is an old lineage, printing experiment details...", old_lineages_txt)
                        log(exp, old_lineages_txt)
                        log('------------------------', old_lineages_txt)

                        log('------------------------', old_lineages_comparison)
                        log(f'Lineage: {case}', old_lineages_comparison)
                        log(f"GT Lineages: {' + '.join(alias_key.get(case, 'N/A'))}", old_lineages_comparison)
                        log(f'NatComm Paper:', old_lineages_comparison)
                        log(f'This Experiment: {recombinhunt_candidates}', old_lineages_comparison)
                        log('------------------------', old_lineages_comparison)
                        
                        parent_lineage_list = cl.contributing_to(case)
                        breakpoints = BreakpointsLocation.all_breakpoints(case)
                        n_breakpoints = BreakpointsLocation.breakpoints_num(case)

                        ca = CaseAnalysis(exp, case, N_seq, case_n, group_name,
                                            lh, cl,
                                            parent_lineage_list, breakpoints)
                        ca.print_case_details(old_lineages_html)

                    else:
                        log('------------------------', new_lineages_txt)
                        log(f"{case} is a new lineage, printing experiment details...", new_lineages_txt)
                        log(exp, new_lineages_txt)
                        log('------------------------', new_lineages_txt)

                        log('------------------------', new_lineages_comparison)
                        log(f'Lineage: {case}', new_lineages_comparison)
                        log(f"GT Lineages: {' + '.join(alias_key.get(case, 'N/A'))}", new_lineages_comparison)
                        log(f'This Experiment: {recombinhunt_candidates}', new_lineages_comparison)
                        log('------------------------', new_lineages_comparison)

                        lh = exp.lh
                        ca = CaseAnalysis(exp, case, N_seq, case_n, group_name,
                                          lh)
                        ca.print_case_details(new_lineages_html)
                                          

                    # # print()

                    # #COMPARE EXPERIMENT RESULT WITH GROUND TRUTH DATA
                    # N_seq = f"(75%) {N_seq}"
                    # # ca = CaseAnalysis(exp, case, N_seq, case_n, group_name,
                    # #                  parent_lineage_list, breakpoints,
                    # #                  lh, cl)

                    # ca = CaseAnalysis(exp, case, N_seq, case_n, group_name,
                    #                  lh, cl)

                    # ca.print_case_details(detailed_cases_outfile)

                    # issues[ca.get_issue()].append(case)

                    # tabella_riassuntiva.append(ca.analysis_table_row())

                except Exception as e:
                        print("ERROR", case, end=" ")
                        
                        ### ---debug the error output
                        #print('breakpoints: ', breakpoints)
                        
                        
                        try:
                            #print("Last analysed case", tabella_riassuntiva[-1][1], "case n° (1-based)", tabella_riassuntiva[-1][0])
                            pass
                        except IndexError:
                            pass
                        raise e


  2%|▏         | 2/98 [00:00<00:43,  2.20it/s]

Analysing case XAD (1/98) with 14 sequences, group: 1BP mid


  3%|▎         | 3/98 [00:01<00:57,  1.66it/s]

Analysing case XAE (2/98) with 15 sequences, group: 1BP mid


  4%|▍         | 4/98 [00:02<01:01,  1.53it/s]

...

Analysing case XEM (96/98) with 3 sequences, group: undefined


100%|██████████| 98/98 [01:15<00:00,  1.35it/s]

Analysing case XEN (97/98) with 11 sequences, group: undefined


100%|██████████| 98/98 [01:16<00:00,  1.28it/s]

Analysing case XEP (98/98) with 3 sequences, group: undefined
